In [1]:
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import relationship
from sqlalchemy import and_
from sqlalchemy.orm import declarative_base

In [2]:
#NOTE-InitializeFunctions :

# engine.dispose()
# sess.close()
# base.registry.dispose()
# base.metadata.clear()

In [3]:
engine = create_engine('sqlite:///:memory:', echo=False)
base = declarative_base()
Session = sessionmaker(engine)
sess = Session()

In [4]:
class Post(base):
    __tablename__ = 'POST'
    __table_args__ = { 'extend_existing': True }
    pk = Column('PK', Integer, primary_key=True)
    content = Column('CONTENT', Text)
    tags = relationship('Tags', back_populates='post', uselist=True)
    
    def add_tags(self, *tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(HashTag).filter(HashTag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = HashTag(name = tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1=self.pk, fk2=tag.pk))
            sess.commit()
            tag.cnt_increment()
            
    def del_tags(self, *tags):
        tagList = list()
        for tag in tags:
            rst = sess.query(HashTag).filter(HashTag.name == tag)
            if rst.count() <= 0:
                continue
            else:
                res = rst.one()
                tagToDelete = sess.query(Tags).where(and_(Tags.fk1 == post3.pk, Tags.fk2 == res.pk)).one()
                sess.delete(tagToDelete)
                sess.commit()
                res.cnt_decrement()
    
    def __repr__(self):
        return f'PK:{self.pk}, CONTENT:{self.content},, TAG_LEN:{len(self.tags)}'
    
class HashTag(base):
    __tablename__ = 'HASHTAG'
    pk = Column('PK', Integer, primary_key=True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag', uselist=True)
    
    def cnt_increment(self):
        self.cnt = self.cnt + 1
        
    def cnt_decrement(self):
        self.cnt = self.cnt - 1
    
    def del_hashtag(self):
        tagsToDel = sess.query(Tags).where(Tags.fk2 == self.pk)
        for tag in tagsToDel:
            sess.delete(tag)
            sess.commit()
        hashTagToDel = sess.query(HashTag).where(HashTag.pk == self.pk)
        sess.delete(hashTagToDel)
    
    
    def __repr__(self):
        return f'PK:{self.pk}, NAME:{self.name}, POSTLEN:{len(self.posts)}, COUNT:{self.cnt}'
    
class Tags(base):
    __tablename__ = 'TAGS'
    pk = Column('PK', Integer, primary_key=True)
    fk1 = Column('FK1', Integer, ForeignKey('POST.PK'))
    fk2 = Column('FK2', Integer, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('HashTag', back_populates='posts')
    
    def del_tag(self):
        tagToDel = sess.query(Tags).where(Tags.pk == self.pk).one()
        tagToDel.hashtag.cnt_decrement()
        sess.delete(tagToDel)
        sess.commit()
        
    
    def __repr__(self):
        return f'PK{self.pk} : {self.fk1} -- {self.fk2}'

In [5]:
base.metadata.create_all(engine)

In [6]:
def add_post(contentParam: str, *hashTags: [str]):
    newPost = Post(content = contentParam)
    sess.add(newPost)
    sess.commit()
    newPost.add_tags(*hashTags)

***
**TESTCASE01 : ADD POST -->**

In [7]:
add_post("테스트내용1","해시태그1", "해시태그2", "해시태그3")

In [8]:
add_post("테스트내용2","해시태그3", "해시태그4", "해시태그5")

In [9]:
add_post("테스트내용3","해시태그1")

In [10]:
sess.query(Post).all()

[PK:1, CONTENT:테스트내용1,, TAG_LEN:3,
 PK:2, CONTENT:테스트내용2,, TAG_LEN:3,
 PK:3, CONTENT:테스트내용3,, TAG_LEN:1]

In [11]:
sess.query(HashTag).all()

[PK:1, NAME:해시태그1, POSTLEN:2, COUNT:2,
 PK:2, NAME:해시태그2, POSTLEN:1, COUNT:1,
 PK:3, NAME:해시태그3, POSTLEN:2, COUNT:2,
 PK:4, NAME:해시태그4, POSTLEN:1, COUNT:1,
 PK:5, NAME:해시태그5, POSTLEN:1, COUNT:1]

**<-- TESTCASE 01 END**
***

In [12]:
def modif_post(postPK: int, newContent: str):
    postToModif = sess.query(Post).where(Post.pk == postPK)
    if postToModif.count() > 0:
        postToModif.one().content = newContent
        sess.commit()
    else:
        pass

***
**TESTCASE 02 : MODIFY POST -->**

In [13]:
modif_post(1,"새로운테스트내용1")

In [14]:
sess.query(Post).all()

[PK:1, CONTENT:새로운테스트내용1,, TAG_LEN:3,
 PK:2, CONTENT:테스트내용2,, TAG_LEN:3,
 PK:3, CONTENT:테스트내용3,, TAG_LEN:1]

**<-- TESTCASE 02 END**
***

In [15]:
def del_post(postPK: int):
    postToDel = sess.query(Post).where(Post.pk == postPK)
    for tag in postToDel.one().tags:
        tag.del_tag()
    sess.delete(postToDel.one())
    sess.commit()

***
**TESTCASE 03 : DELETE POST -->**

In [16]:
del_post(1)

In [17]:
sess.query(Post).all()

[PK:2, CONTENT:테스트내용2,, TAG_LEN:3, PK:3, CONTENT:테스트내용3,, TAG_LEN:1]

In [18]:
sess.query(HashTag).all()

[PK:1, NAME:해시태그1, POSTLEN:1, COUNT:1,
 PK:2, NAME:해시태그2, POSTLEN:0, COUNT:0,
 PK:3, NAME:해시태그3, POSTLEN:1, COUNT:1,
 PK:4, NAME:해시태그4, POSTLEN:1, COUNT:1,
 PK:5, NAME:해시태그5, POSTLEN:1, COUNT:1]

In [19]:
del_post(2)

In [20]:
sess.query(Post).all()

[PK:3, CONTENT:테스트내용3,, TAG_LEN:1]

In [21]:
sess.query(HashTag).all()

[PK:1, NAME:해시태그1, POSTLEN:1, COUNT:1,
 PK:2, NAME:해시태그2, POSTLEN:0, COUNT:0,
 PK:3, NAME:해시태그3, POSTLEN:0, COUNT:0,
 PK:4, NAME:해시태그4, POSTLEN:0, COUNT:0,
 PK:5, NAME:해시태그5, POSTLEN:0, COUNT:0]

**<-- TESTCASE 03 END**
***